# 크롤링을 이용한 서울시 스타벅스 매장 목록 데이터 생성

In [1]:
#
!pip install selenium

     |████████████████████████████████| 911kB 5.4MB/s 


In [2]:
!apt-get update # apt install을 정확히 실행하기 위해 업데이트 
!apt install chromium-chromedriver 
!cp /usr/lib/chromium-browser/chromedriver /usr/bin 
import sys 
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') 

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://se

In [3]:
# 예제 6-1  라이브러리 임포트
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import quote_plus
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys

In [4]:
chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument('--headless') 
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-dev-shm-usage')

In [5]:
# 예제 6-2  webdriver 실행 후 스타벅스의 지역별 매장 검색 화면에 접속
#driver = webdriver.Chrome('c:/playwithdata/chromedriver.exe')
driver = webdriver.Chrome('chromedriver',options=chrome_options)
# driver = webdriver.Chrome('../chromedriver')
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
#url='http://naver.com'
driver.get(url)

In [6]:
#팝업창 없애기
close_btn = 'body > div.holiday_notice.addBtnHoliday > p > a'
driver.find_element_by_css_selector(close_btn).click()

NoSuchElementException: ignored

In [7]:
# 예제 6-3 webdriver로 ‘서울’ 버튼 요소를 찾아 클릭
#seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul_btn).click()

In [8]:
# 예제 6-4  webdriver로 ‘전체’ 버튼 요소를 찾아 클릭
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all_btn).click()

In [9]:
# 예제 6-5  BeautifulSoup으로 HTML 파서 만들기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [10]:
# 예제 6-6  select()를 이용해 원하는 HTML 태그를 모두 찾아오기
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

540


In [11]:
# 예제 6-7  태그 문자열 살펴보기
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [12]:
# 예제 6-8 스타벅스 매장 정보 샘플 확인
startbucks_store = starbucks_soup_list[0]
name = startbucks_store.select('strong')[0].text.strip()
lat = startbucks_store['data-lat'].strip()
lng = startbucks_store['data-long'].strip()
store_type = startbucks_store.select('i')[0]['class'][0][4:]
address = str(startbucks_store.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(startbucks_store.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

print(name)         # 매장명
print(lat)          # 위도
print(lng)          # 경도
print(store_type)   # 매장 타입
print(address)      # 주소
print(tel)          # 전화번호

역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [13]:
# 예제 6-9 서울시 스타벅스 매장 목록 데이터 만들기
starbucks_list = []
for item in starbucks_soup_list:
    name = item.select('strong')[0].text.strip();
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0][4:]
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    
    starbucks_list.append( [ name, lat, lng, store_type, address, tel])

In [15]:
# 예제 6-10  pandas의 데이터프레임 생성
columns = ['매장명','위도','경도','매장타입', '주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns = columns)
seoul_starbucks_df.tail()

,매장명,위도,경도,매장타입,주소,전화번호
535,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232
536,상봉역,37.59689,127.08647,general,"서울특별시 중랑구 망우로 307, ,3,4번지 (상봉동)",1522-3232
537,묵동이마트,37.613433,127.077484,general,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",1522-3232
538,묵동,37.615368,127.076633,general,서울특별시 중랑구 동일로 952,1522-3232
539,중화역,37.60170912407773,127.07841136432036,general,"서울특별시 중랑구 봉화산로 35 1,2층",1522-3232


In [16]:
# 예제 6-11  데이터프레임의 요약 정보 확인
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     540 non-null    object
 1   위도      540 non-null    object
 2   경도      540 non-null    object
 3   매장타입    540 non-null    object
 4   주소      540 non-null    object
 5   전화번호    540 non-null    object
dtypes: object(6)
memory usage: 25.4+ KB


In [17]:
# 예제 6-12 엑셀로 저장
seoul_starbucks_df.to_excel('seoul_starbucks_list.xlsx', index=False)